## Requirments

In [1]:
import numpy as np
import pandas as pd

## Shared or not shared?

When creating a new dataframe out of an existing dataframe, is column data shared or not?

The answer to that question seems to depend on the version of `pandas`.

In [2]:
pd.__version__

'2.3.3'

## Pandas 2.x

To see what happens when you modify a dataframe from which another is derived, you can start by creating a large dataframe.

In [3]:
data = pd.DataFrame({
    'column1': np.random.uniform(-1_000.0, 1_000.0, size=100_000),
    'column2': np.random.uniform(-1_000.0, 1_000.0, size=100_000),
    'column3': np.random.uniform(-1_000, 1_000, size=100_000).astype(np.int64),
    'column4': np.random.uniform(-1_000, 1_000, size=100_000).astype(np.int64),
})

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   column1  100000 non-null  float64
 1   column2  100000 non-null  float64
 2   column3  100000 non-null  int64  
 3   column4  100000 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 3.1 MB


The data has the statistical properties you would expect from a uniform distribution.

In [5]:
data.describe()

,column1,column2,column3,column4
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.602956,-0.641311,0.787490,0.506170
std,576.655889,576.314473,576.188924,575.342083
min,-999.959910,-999.992564,-999.000000,-999.000000
25%,-498.711295,-499.539365,-496.000000,-498.000000
50%,1.613398,1.272861,1.500000,0.500000
75%,498.793140,496.132510,500.000000,498.000000
max,999.998920,999.995667,999.000000,999.000000


When you create a new dataframe with half the number of rows as the original dataset, the question is whether or not the two dataframes share data.

In [6]:
data2 = data.iloc[:50_000]

In [7]:
data2.describe()

,column1,column2,column3,column4
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,-0.806390,-2.411274,1.028800,0.585800
std,576.391766,576.943889,576.134753,575.058659
min,-999.959910,-999.976965,-999.000000,-999.000000
25%,-499.906753,-502.311204,-493.000000,-499.000000
50%,0.363471,-2.524545,2.000000,1.000000
75%,496.309675,494.880591,500.000000,498.000000
max,999.998920,999.992495,999.000000,999.000000


To ascertain this, you can modify one of the dataframes, say `data` in place, and check whether the other dataframe is affected.  For insstance, yu can clip the values of the first column to the interval $[-500, 500]$.

In [8]:
data['column1'].clip(-500.0, 500.0, inplace=True)

/tmp/ipykernel_10532/3787905307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['column1'].clip(-500.0, 500.0, inplace=True)


In [9]:
data.describe()

,column1,column2,column3,column4
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.936049,-0.641311,0.787490,0.506170
std,407.737463,576.314473,576.188924,575.342083
min,-500.000000,-999.992564,-999.000000,-999.000000
25%,-498.711295,-499.539365,-496.000000,-498.000000
50%,1.613398,1.272861,1.500000,0.500000
75%,498.793140,496.132510,500.000000,498.000000
max,500.000000,999.995667,999.000000,999.000000


In [10]:
data2.describe()

,column1,column2,column3,column4
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,0.129833,-2.411274,1.028800,0.585800
std,407.710515,576.943889,576.134753,575.058659
min,-500.000000,-999.976965,-999.000000,-999.000000
25%,-499.906753,-502.311204,-493.000000,-499.000000
50%,0.363471,-2.524545,2.000000,1.000000
75%,496.309675,494.880591,500.000000,498.000000
max,500.000000,999.992495,999.000000,999.000000


In [11]:
data is data2

False

Indeed, the first column of `data2` was affected, its minimum and maximem values aore no longer approximately -1000 and 1000 as before, but rather -500 and 500 respectively.

This may surprise you, and is of course a potential source of subtle bugs.  It is a trade-off, on the one hand sharing data between related dataframes saves memory, but also increases performance, since copying data is expensive in terms of execution time.  On the other hand, it means you perceive "spooky action at a distance" since operations on one dataframe may affect a related dataframe.

The developers of pandas want to ensure that an operation on a dataframe never affects another dataframe and have introduced Copy-on-Write in pandas 1.5.x.  It is a "mode" that can be set via pandas `options` module.  Pandas 2.x will generate some warnings for obvious cases when Copy-on-Write is violated, so that you can adapt your code accordingly.

The warning displayed when you executed the `clip` operation in-place is an illustration of that.

## Pandas 3.x

In pandas 3.x, Copy-on-Write will be the only mode that is available, it will ensure that only a single dataframe or series is affected by an operation.  When using pandas 2.x, you can optionally switch on this semantics already.

In [12]:
pd.options.mode.copy_on_write = True

You can replay the same scenario as before.

In [13]:
data = pd.DataFrame({
    'column1': np.random.uniform(-1_000.0, 1_000.0, size=100_000),
    'column2': np.random.uniform(-1_000.0, 1_000.0, size=100_000),
    'column3': np.random.uniform(-1_000, 1_000, size=100_000).astype(np.int64),
    'column4': np.random.uniform(-1_000, 1_000, size=100_000).astype(np.int64),
})

In [14]:
data2 = data.iloc[:50_000]

In [15]:
data2.describe()

,column1,column2,column3,column4
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,2.851120,-0.056905,1.032220,-0.735960
std,578.724236,576.210641,574.166908,576.784099
min,-999.976479,-999.957931,-999.000000,-999.000000
25%,-499.984268,-499.984703,-494.000000,-499.250000
50%,1.331581,-0.727104,0.000000,-2.000000
75%,507.906292,496.411519,497.000000,495.000000
max,999.956974,999.901959,999.000000,999.000000


In [16]:
data['column1'].clip(-500.0, 500.0, inplace=True)

/tmp/ipykernel_10532/3787905307.py:1: ChainedAssignmentError: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
When using the Copy-on-Write mode, such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object.


  data['column1'].clip(-500.0, 500.0, inplace=True)


This time around, thanks to setting the `copy_on_write` mode to `True`, an error is raised, and you can verify that `data2` is unaffected.

In [17]:
data2.describe()

,column1,column2,column3,column4
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,2.851120,-0.056905,1.032220,-0.735960
std,578.724236,576.210641,574.166908,576.784099
min,-999.976479,-999.957931,-999.000000,-999.000000
25%,-499.984268,-499.984703,-494.000000,-499.250000
50%,1.331581,-0.727104,0.000000,-2.000000
75%,507.906292,496.411519,497.000000,495.000000
max,999.956974,999.901959,999.000000,999.000000


Of course, the orginal dataframe `data` is also unmodified.

In [18]:
data.describe()

,column1,column2,column3,column4
count,100000.000000,100000.000000,100000.000000,100000.00000
mean,3.105649,-1.964748,-1.940910,-1.55023
std,578.037911,576.648688,575.071945,576.57867
min,-999.988675,-999.997376,-999.000000,-999.00000
25%,-497.744633,-502.995143,-498.000000,-500.00000
50%,0.822980,-3.462880,-3.000000,-2.00000
75%,506.518015,496.792593,495.000000,496.00000
max,999.980718,999.901959,999.000000,999.00000


Following the advice of the warning/error message, you can execute the `clip` operation on the dataframe, specifying the clip values for each column, where `None` will not clip any values for that column.

In [19]:
data.clip([-500.0, *[None]*3], [500.0, *[None]*3], inplace=True)

You can verify that `data2` is unaffected, as intended.

In [20]:
data2.describe()

,column1,column2,column3,column4
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,2.851120,-0.056905,1.032220,-0.735960
std,578.724236,576.210641,574.166908,576.784099
min,-999.976479,-999.957931,-999.000000,-999.000000
25%,-499.984268,-499.984703,-494.000000,-499.250000
50%,1.331581,-0.727104,0.000000,-2.000000
75%,507.906292,496.411519,497.000000,495.000000
max,999.956974,999.901959,999.000000,999.000000


Of course, the values in the first column of `data` are clipped as expected.

In [21]:
data.describe()

,column1,column2,column3,column4
count,100000.000000,100000.000000,100000.000000,100000.00000
mean,1.694298,-1.964748,-1.940910,-1.55023
std,408.735909,576.648688,575.071945,576.57867
min,-500.000000,-999.997376,-999.000000,-999.00000
25%,-497.744633,-502.995143,-498.000000,-500.00000
50%,0.822980,-3.462880,-3.000000,-2.00000
75%,500.000000,496.792593,495.000000,496.00000
max,500.000000,999.901959,999.000000,999.00000


Alternatively, the values can also be clipped as follows for the second column.

In [22]:
data['column2'] = data['column2'].clip(-500.0, 500.0)

In [23]:
data.describe()

,column1,column2,column3,column4
count,100000.000000,100000.000000,100000.000000,100000.00000
mean,1.694298,-1.260658,-1.940910,-1.55023
std,408.735909,408.097212,575.071945,576.57867
min,-500.000000,-500.000000,-999.000000,-999.00000
25%,-497.744633,-500.000000,-498.000000,-500.00000
50%,0.822980,-3.462880,-3.000000,-2.00000
75%,500.000000,496.792593,495.000000,496.00000
max,500.000000,500.000000,999.000000,999.00000


Dataframe `data2` is unaffected as well.

In [24]:
data2.describe()

,column1,column2,column3,column4
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,2.851120,-0.056905,1.032220,-0.735960
std,578.724236,576.210641,574.166908,576.784099
min,-999.976479,-999.957931,-999.000000,-999.000000
25%,-499.984268,-499.984703,-494.000000,-499.250000
50%,1.331581,-0.727104,0.000000,-2.000000
75%,507.906292,496.411519,497.000000,495.000000
max,999.956974,999.901959,999.000000,999.000000


### Performance?

You can check which of the two alternatives has the better performance for in-place operations.

In [25]:
nr_rows = 100_000

In [26]:
data = pd.DataFrame({
    'column1': np.random.uniform(-1_000.0, 1_000.0, size=nr_rows),
    'column2': np.random.uniform(-1_000.0, 1_000.0, size=nr_rows),
    'column3': np.random.uniform(-1_000, 1_000, size=nr_rows).astype(np.int64),
    'column4': np.random.uniform(-1_000, 1_000, size=nr_rows).astype(np.int64),
})

In [27]:
%%timeit
data.clip([-500.0, None, None, None], [500.0, None, None, None], inplace=True)

7.99 ms ± 722 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%%timeit
data['column1'] = data['column1'].clip(-500.0, 500.0)

1 ms ± 54.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
nr_rows = 10_000_000

In [30]:
data = pd.DataFrame({
    'column1': np.random.uniform(-1_000.0, 1_000.0, size=nr_rows),
    'column2': np.random.uniform(-1_000.0, 1_000.0, size=nr_rows),
    'column3': np.random.uniform(-1_000, 1_000, size=nr_rows).astype(np.int64),
    'column4': np.random.uniform(-1_000, 1_000, size=nr_rows).astype(np.int64),
})

In [31]:
%%timeit
data.clip([-500.0, None, None, None], [500.0, None, None, None], inplace=True)

954 ms ± 81.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%%timeit
data['column1'] = data['column1'].clip(-500.0, 500.0)

64.5 ms ± 8.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


As you can see, assigning the column to perform an in-place operations is significantly faster.

## Conclusion

If your code relies on "spooky action at a distance" you have to prepare for the release of pandas 3.x, since your code will no longer work and generate errors.  You can do so by enabling Copy-on-Write via the `options` and fix the resulting issues.

In general, it is good practice to enable Copy-on-Write for development of new code as well, since you will be sure that when pandas 3.x is release, your code will at least work without issues related to Copy-on-Write.

You can find more information and examples in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/copy_on_write.html).